# Problem 1



In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

import statsmodels.api as sm

import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from pylab import rcParams
from sklearn.model_selection import KFold

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
cmap1 = matplotlib.cm.get_cmap('summer')
import seaborn as sb
%matplotlib inline
%matplotlib notebook
rcParams['figure.figsize'] = 10, 8
sb.set_style('whitegrid')

## (a)

In [56]:
autodf = pd.read_csv('Auto.csv', delimiter = ',', header = 0, \
                     names = ['mpg', 'cyl', 'dspl', 'hpwr', 'wgt', 'accl', 'yr', 'org', 
                              'name'], na_values = '?')
autodf['hpwr'] = autodf['hpwr'].apply(pd.to_numeric, errors = 'coerce')
mpg_median = autodf['mpg'].median()
autodf['mpg_high'] = autodf['mpg'] >= mpg_median
autodf['mpg_high'] = autodf['mpg_high'].astype(int)
autodf.head()

,mpg,cyl,dspl,hpwr,wgt,accl,yr,org,name,const,mpg_high
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,1.0,0
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,1.0,0
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,1.0,0
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,1.0,0
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,1.0,0


In [103]:
LogitModel1 = sm.Logit(endog = autodf['mpg_high'], exog = autodf[['cyl', 'dspl', 'hpwr',\
                                                  'wgt', 'accl', 'yr', 'org']], missing = 'drop')
result1 = LogitModel1.fit(fit_intercept = True)
result1.summary()

Optimization terminated successfully.
         Current function value: 0.213308
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               mpg_high   No. Observations:                  392
Model:                          Logit   Df Residuals:                      385
Method:                           MLE   Df Model:                            6
Date:                Sun, 04 Mar 2018   Pseudo R-squ.:                  0.6923
Time:                        22:20:23   Log-Likelihood:                -83.617
converged:                       True   LL-Null:                       -271.71
                                        LLR p-value:                 3.654e-78
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
cyl           -0.2029      0.402     -0.505      0.613      -0.990       0.584
dspl          -0.0018      0.011     -0.167      0.867      -0.024       0.020
hpwr          -0.0761      0.021     -3.643      0.000      -0.117      -0.035
wgt           -0.0029      0.001     -3.047      0.002      -0.005      -0.001
accl          -0.2304      0.116     -1.982      0.047      -0.458      -0.003
yr             0.2637      0.043      6.182      0.000       0.180       0.347
org            0.2602      0.336      0.773      0.439      -0.399       0.920
==============================================================================

Possibly complete quasi-separation: A fraction 0.14 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [104]:
autodf.dropna(inplace = True)
X = autodf[['const', 'cyl', 'dspl', 'hpwr','wgt', 'accl', 'yr', 'org']]
Xvars = X.values
y = autodf['mpg_high']
yvals = y.values
k = 4
kf_log = KFold(n_splits=4, shuffle=True, random_state=15)

kf_log.get_n_splits(Xvars)

MSE_vec_kf = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf_log.split(Xvars):
    # print("TRAIN:", train_index, "TEST:", test_index)
    print('k index=', k_ind)
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    LogReg = LogisticRegression(fit_intercept = True)
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    print(classification_report(y_test, y_pred))
    k_ind += 1

MSE_kf1 = MSE_vec_kf.mean()
MSE_kf_std1 = MSE_vec_kf.std()


k index= 0
MSE for test set 0  is 0.0918367346939
             precision    recall  f1-score   support

          0       0.94      0.89      0.92        55
          1       0.87      0.93      0.90        43

avg / total       0.91      0.91      0.91        98

k index= 1
MSE for test set 1  is 0.102040816327
             precision    recall  f1-score   support

          0       0.88      0.91      0.90        47
          1       0.92      0.88      0.90        51

avg / total       0.90      0.90      0.90        98

k index= 2
MSE for test set 2  is 0.132653061224
             precision    recall  f1-score   support

          0       0.85      0.87      0.86        45
          1       0.88      0.87      0.88        53

avg / total       0.87      0.87      0.87        98

k index= 3
MSE for test set 3  is 0.102040816327
             precision    recall  f1-score   support

          0       0.95      0.84      0.89        49
          1       0.85      0.96      0.90        4

In [105]:
print('test estimate MSE k-fold=', MSE_kf1,
      'test estimate MSE standard err=', MSE_kf_std1)
print('The error rate for j = 0 is:', 1 - (0.94 + 0.88 + 0.85 + 0.95) / 4)
print('The error rate for j = 1 is:', 1 - (0.87 + 0.92 + 0.88 + 0.85) / 4)

test estimate MSE k-fold= 0.107142857143 test estimate MSE standard err= 0.015306122449
The error rate for j = 0 is: 0.09499999999999997
The error rate for j = 1 is: 0.12


## (b)

In [93]:
from sklearn.ensemble import RandomForestClassifier
Xb = autodf[['cyl', 'dspl', 'hpwr','wgt', 'accl', 'yr', 'org']]
Xvarsb = Xb.values
auto_tree = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True,
                           oob_score=True, random_state=25)

auto_tree.fit(Xvarsb, yvals)
y_oob = auto_tree.oob_decision_function_
y_pred2 = []
for item in y_oob: 
    y_pred2.append(item[1])

In [94]:
y_df = pd.DataFrame({'ypred': y_pred2, 'yvals': yvals})

In [98]:
y_df['ypred'] = y_df['ypred'].apply(lambda x:1 if x>= 0.5 else 0)
MSE2 = mean_squared_error(y_df['yvals'], y_df['ypred'])
ypred_0 = y_df[y_df['ypred'] < 0.5]
MSE_0 = mean_squared_error(ypred_0['yvals'], ypred_0['ypred'])
ypred_1 = y_df[y_df['ypred'] >= 0.5]
MSE_1 = mean_squared_error(ypred_1['yvals'], ypred_1['ypred'])
print(classification_report(yvals, y_df['ypred']))
print('MSE=', MSE2)
print('MSE for class 0 is:', MSE_0)
print('MSE for class 1 is:', MSE_1)

             precision    recall  f1-score   support

          0       0.94      0.91      0.93       196
          1       0.92      0.94      0.93       196

avg / total       0.93      0.93      0.93       392

MSE= 0.0714285714286
MSE for class 0 is: 0.0578947368421
MSE for class 1 is: 0.0841584158416


## (c)

In [19]:
from sklearn import svm
from sklearn.datasets import make_blobs
from sklearn import datasets
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
C = 1.0  # SVM regularization parameter
model_svm = svm.SVC(kernel='rbf', gamma=0.2, C=C)

In [101]:
k = 4
kf_svm = KFold(n_splits=4, shuffle=True, random_state=15)

kf_svm.get_n_splits(Xvarsb)
SVM_error_0 = np.zeros(k)
SVM_error_1 = np.zeros(k)
MSE_vec_kf = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf_svm.split(Xvars):
    # print("TRAIN:", train_index, "TEST:", test_index)
    print('k index=', k_ind)
    X_train, X_test = Xvarsb[train_index], Xvarsb[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    model_svm = svm.SVC(kernel='rbf', gamma=0.2, C=1.0)
    model_svm.fit(X_train, y_train)
    y_pred = model_svm.predict(X_test)
    MSE_vec_kf[k_ind] = (y_test != y_pred).mean()
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    print(classification_report(y_test, y_pred))
    k_ind += 1

MSE_kf3 = MSE_vec_kf.mean()
MSE_kf_std3 = MSE_vec_kf.std()

k index= 0
MSE for test set 0  is 0.540816326531
             precision    recall  f1-score   support

          0       1.00      0.04      0.07        55
          1       0.45      1.00      0.62        43

avg / total       0.76      0.46      0.31        98

k index= 1
MSE for test set 1  is 0.520408163265
             precision    recall  f1-score   support

          0       0.48      1.00      0.65        47
          1       0.00      0.00      0.00        51

avg / total       0.23      0.48      0.31        98

k index= 2
MSE for test set 2  is 0.520408163265
             precision    recall  f1-score   support

          0       0.47      1.00      0.64        45
          1       1.00      0.04      0.07        53

avg / total       0.76      0.48      0.33        98

k index= 3
MSE for test set 3  is 0.448979591837
             precision    recall  f1-score   support

          0       0.53      1.00      0.69        49
          1       1.00      0.10      0.19        49

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [102]:
print("The error rate for class 0 is:", 1 - (1.00 + 0.48 + 0.47 + 0.53)/4)
print("The error rate for class 1 is:", 1 - (0.45 + 0.00 + 1.00 + 1.00)/4)
print("The average MSE for the dataset is:", MSE_kf3, "std =", MSE_kf_std3)

The error rate for class 0 is: 0.38
The error rate for class 1 is: 0.38749999999999996
The average MSE for the dataset is: 0.507653061224 std = 0.0348846794163


## (d)

In [100]:
print("Logit Regressor: MSE = {}, \
error rate for class 0 is: {}, \
error rate for class 1 is : {}".format(MSE_kf1, 0.09499999999999997, 0.12))
print("Random Forest Classifier: MSE = {}, \
error rate for class 0 is: {}, \
error rate for class 1 is : {}".format(MSE2, MSE_0, MSE_1))
print("SVC: MSE = {}, \
error rate for class 0 is: {}, \
error rate for class 1 is : {}".format(MSE_kf3, 0.38, 0.38749999999999996))

Logit Regressor: MSE = 0.10714285714285715, error rate for class 0 is: 0.09499999999999997, error rate for class 1 is : 0.12
Random Forest Classifier: MSE = 0.07142857142857142, error rate for class 0 is: 0.05789473684210526, error rate for class 1 is : 0.08415841584158416
SVC: MSE = 0.5076530612244898, error rate for class 0 is: 0.38, error rate for class 1 is : 0.38749999999999996


In [99]:
from sklearn.model_selection import cross_val_score
print('cross val score for Random Forest Classifier is:', cross_val_score(auto_tree, X, y, cv = 4))
print('cross val score for SVC is:', cross_val_score(model_svm, X, y, cv = 4))

cross val score for Random Forest Classifier is: [ 0.70408163  0.87755102  0.89795918  0.76530612]
cross val score for SVC is: [ 0.5         0.51020408  0.51020408  0.5       ]


By comparing MSE, we could see that Random Forest Classifier has the smallest MSE(0.07143), SVC has the largest MSE(0.50765), and the Logit Regressor in between(0.10714). We can see that Random Forest has the lowest error rates for both class 0 and class 1 (mpg_high = 0 and mpg_high = 1). That there is a mixture of categorical and continuous features might be the reason why Random Forest Classifier is better here. 